In [ ]:
!pip install datasets
!pip install transformers
!pip install apache-beam
!pip install tiktoken
!pip install langchain
!pip install openai
!pip install pinecone
!pip install pinecone-client

  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
ERROR: Operation cancelled by user
ERROR: Operation cancelled by user
ERROR: Could not find a version that satisfies the requirement pinecone (from versions: none)
ERROR: No matching distribution found for pinecone


In [ ]:
import tiktoken

tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken_len("hello I am a chunk of text and using the tiktoken_len function "
             "we can find the length of this chunk of text in tokens")

26

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
chunks = text_splitter.split_text(data[6]['text'])[:3]
chunks

['Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.\n\nEarly life and family \nAlan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.\n\nEducation \nTuring went to St. Michael\'s, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.\n"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.\n\nThe Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.\n\nEducated in Dublin at Alexandra School and College; on October 1st 1907 she married Julius Mathison Turing, latter son o

In [ ]:
tiktoken_len(chunks[0]), tiktoken_len(chunks[1]), tiktoken_len(chunks[2])

(299, 323, 382)

In [ ]:
# Embedding

!pip install -q sentence-transformers
!pip install -q langchain

from langchain.embeddings import HuggingFaceEmbeddings
import torch

model_name = "sentence-transformers/all-MiniLM-L6-v2" # all-MiniLM-L6-v2: structure of SBert
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is hereeeeeeeeeeee'
]
res = torch.tensor(hf.embed_documents(texts))
len(res),len(res[0])

(2, 384)

In [ ]:
import pinecone


YOUR_API_KEY = getpass("Pinecone API Key:")
YOUR_ENV = input("Pinecone environment:")
index_name = 'langchain-retrieval-augmentation'
pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)



Pinecone API Key:··········
Pinecone environment:us-west4-gcp-free


In [ ]:
pinecone.create_index(
        name="firsttry",
        metric='cosine',
        dimension=len(res[0])  # 1536 dim of text-embedding-ada-002
    )

In [ ]:
index = pinecone.Index("firsttry")

index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
from datasets import load_dataset

data = load_dataset('csv', data_files='/content/output_v2_table.csv', encoding = "unicode_escape")


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data = data['train']
dataset_name = data['Title']

In [ ]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100

texts = []
metadatas = []
total_id = []
for i, record in enumerate(tqdm(data)):
    # first get metadata fields for this record
    metadata = {
        'area': record['Subject Area'],
        'title': record['Title']
    }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record['Description'])
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "Description": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        #embeds = embed.embed_documents(texts)
        embeds = hf.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []
        total_id = total_id + ids


  0%|          | 0/622 [00:00<?, ?it/s]

In [ ]:
len(total_id)

621

In [ ]:
if len(texts) > 0:
    #ids = [i for i in range(len(texts))]
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = hf.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))
    total_id = total_id + ids

In [ ]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 621}},
 'total_vector_count': 621}

In [ ]:
index = pinecone.Index("firsttry")

In [ ]:
index.fetch([total_id[3]])

{'namespace': '',
 'vectors': {'f3f02945-2346-4b38-a0cd-3fea6cd39cd6': {'id': 'f3f02945-2346-4b38-a0cd-3fea6cd39cd6',
                                                      'metadata': {'Description': 'This '
                                                                                  'data '
                                                                                  'set '
                                                                                  'contains '
                                                                                  'weighted '
                                                                                  'census '
                                                                                  'data '
                                                                                  'extracted '
                                                                                  'from '
                                                          

In [ ]:

test_descrip=[' Predict whether income exceeds  census data. Adult.']
vector = hf.embed_documents(test_descrip)
result = index.query(
  vector,
  top_k=1,
  include_values=True
)



In [ ]:
indexes = total_id.index(result['matches'][0]['id'])
print(dataset_name[indexes])


Census Income


In [ ]:
pinecone.describe_index("firsttry")

IndexDescription(name='firsttry', metric='cosine', replicas=1, dimension=384.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [ ]:
query = "who was Benito Mussolini?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

NameError: ignored